In [ ]:
from bs4 import BeautifulSoup  # Importa BeautifulSoup 
import pandas as pd  # Importa la biblioteca pandas y la renombra como pd
import requests  # Importa la biblioteca requests
import re  # Importa la biblioteca re

pd.set_option('display.max_columns', None)  # Configura pandas para mostrar todas las columnas de un DataFrame al imprimir

url = "https://moviesdatabase.p.rapidapi.com/titles"  # ENDPOINT

headers = {
    "X-RapidAPI-Key": "b0262d2945msh1f1577867db30c6p12fca9jsnc5eb4666d6bb",  # Clave de API para autenticación
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"  # Host de la API 
}

def llamar_pelis(url, headers, year):  # Define una función llamada llamar_pelis 
    datos_paginas = []  # Inicializa una lista vacía para almacenar datos
    base_url = "https://moviesdatabase.p.rapidapi.com"  # URL base para la API
    querystring = {"genre": "Drama", "startYear": year, "titleType": "movie", "endYear": year}  # Parámetros de la consulta
    while url:  # Un bucle while que se ejecuta mientras url no esté vacía
        if not url.startswith('http'):  # Verifica si la url no comienza con 'http'
            url = base_url + url  # Prepara la url completa añadiendo base_url si es necesario
        response = requests.get(url, headers=headers, params=querystring)  # Realiza una solicitud GET a la API
        if response.status_code == 200:  # Comprueba si el estado de la respuesta es 200 (éxito)
            json_pelis = response.json()  # Convierte la respuesta en JSON
            datos_paginas.extend(json_pelis['results'])  # Agrega los resultados al listado de datos_paginas
            url = json_pelis.get('next')  # Actualiza la url con el valor de 'next' si existe, para itinerar por páginas
            querystring = {}  # Limpia los parámetros de la consulta para la siguiente solicitud
        else:
            print("Error en la solicitud:", response.status_code, response.reason)  # Imprime un mensaje de error si falla la solicitud
            return None  # Termina la función retornando None debido a error
    return {"results": datos_paginas}  # Devuelve un diccionario con los datos recopilados

def limpiar_json(datos_json):  # Define una función para procesar y limpiar los datos JSON
    peliculas = []  # Crea una lista vacía para almacenar datos de películas
    for pelicula in datos_json['results']:  # Itera sobre cada película en los resultados
        tipo = pelicula['titleType']['text']  # Extrae el tipo de título de la películ
        nombre = pelicula['titleText']['text']  # Extrae el nombre de la película
        year = pelicula['releaseYear']['year']  # Extrae el año de lanzamiento de la película
        mes = pelicula['releaseDate']['month'] if 'releaseDate' in pelicula and pelicula['releaseDate'] is not None else 'No consta'  # Extrae el mes de lanzamiento o asigna 'No consta'.
        peliculas.append({
            'Tipo': tipo,  # Añade el tipo al diccionario
            'Nombre': nombre,  # Añade el nombre al diccionario
            'Año de Estreno': year,  # Añade el año de estreno al diccionario
            'Mes de Estreno': mes,  # Añade el mes de estreno al diccionario
            'ID': id_pelicula  # Añade el ID al diccionario
        })
    return pd.DataFrame(peliculas)  # Retorna un DataFrame de pandas con los datos de películas

# Proceso iterativo sobre el rango de años
todos_los_datos = []  # Crea una lista vacía para almacenar los DataFrames de cada año
for year in range(2010, 2025):  # Itera sobre cada año desde 2010 hasta 2024
    datos_anuales = llamar_pelis(url, headers, year)  # Llama a la función llamar_pelis para obtener datos del año
    if datos_anuales:  # Si se recibieron datos exitosamente
        df_anual = limpiar_json(datos_anuales)  # Llama a la función limpiar_json para procesar los datos
        todos_los_datos.append(df_anual)  # Añade el DataFrame al listado de todos los datos

# Concatena todos los DataFrames anuales en uno solo
df_final = pd.concat(todos_los_datos, ignore_index=True)  # Usa concatenación para unir todos los DataFrames y reindexa
print(df_final.head(50))  # Imprime los primeros 50 registros del DataFrame final


In [ ]:
# Concatena todos los DataFrames anuales en uno solo
df_final = pd.concat(todos_los_datos, ignore_index=True)
print(df_final.head (50))

In [ ]:
# df_final.to_csv("Peliculas_Head.csv", index=False)